In [1]:
import numpy as np
import os
import tensorflow as tf
from skimage.transform import resize
import matplotlib.pyplot as plt

2025-06-04 12:54:20.918057: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-04 12:54:21.279775: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-06-04 12:54:21.279846: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-06-04 12:54:21.362841: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-04 12:54:21.478643: I tensorflow/core/platform/cpu_feature_guar

In [2]:
# From the previous work, we have the following functions that we have written 

In [3]:
# Write a function for extracting onlt the horizontal images out of all the files in the location

def get_horizontal_images(file_path):
    # get all contents at the file path
    all_files = os.listdir(file_path)
    # get only the image files as we do not need the xml files for the seq2seq model
    only_images = [file for file in all_files if file.split(".")[-1] == 'jpeg']
    # from all the images only choose the horizontal ones
    horizontal_images = [file for file in only_images if file.split('.')[0][-10:] in horizontal_image_list]
    # sort the list
    horizontal_images.sort()
    # return the image list
    return horizontal_images

In [4]:
def create_sub_windows_and_extract_features(folder_path, file, stride = 32, kernel_size = 32):
    # joined image path
    joined_im_path = os.path.join(folder_path, file)
    # read the image
    loaded_im_file = plt.imread(joined_im_path)
    # create subwindows and get prediction
    img_height = loaded_im_file.shape[0]
    img_width = loaded_im_file.shape[1]

    # catch alle extracted features here
    all_extracted_features = []
    # you can also keep track the subwindows here if required - but let's not worry about that for now
    for i in  range(0, img_height, stride):
        for j in range(0, img_width, stride):
            sub_window = loaded_im_file[i: i + kernel_size, j : j + kernel_size,:]
            # resize the subwindow - for 300*300
            sub_window = resize(sub_window, (kernel_size, kernel_size,3))
            # expand the dimension of the window to get predictions from the feature extractor - (make subwindows from shape (32,32,3) to
            # (1,32,32,3))
            sub_window = np.expand_dims(sub_window, axis = 0)
            # extract the features
            predicted_features = feature_extractor_model.predict(sub_window)
            # append the extracted features
            all_extracted_features.append(predicted_features)

    return all_extracted_features  

In [5]:
# The images we use in the task are in the following list

In [6]:
horizontal_image_list = ['2020_08_03', '2020_08_04', '2020_08_06', '2020_08_07', '2020_08_11', '2020_08_12', '2020_08_14', '2020_08_15', '2020_08_17', '2020_08_18', '2020_08_19', '2020_08_21', '2020_08_25', '2020_08_26', '2020_08_27', '2020_08_28', '2020_08_31', '2020_09_02', '2020_09_07', '2020_09_16']

In [7]:
# We also need the fine-tuned model for feature extraction

In [8]:
# load the finetuned gmp model
gmp_model = tf.keras.models.load_model('../../Spring_2024/Bayes_for_comps/TS_bayes_implementation_for_TN/models/trained_gmp_model_dense_32_new.h5')

2025-06-04 12:54:34.444194: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31134 MB memory:  -> device: 0, name: Tesla V100S-PCIE-32GB, pci bus id: 0000:06:00.0, compute capability: 7.0


In [9]:
gmp_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 conv2d (Conv2D)             (None, None, None, 32)    896       
                                                                 
 activation (Activation)     (None, None, None, 32)    0         
                                                                 
 conv2d_1 (Conv2D)           (None, None, None, 32)    9248      
                                                                 
 activation_1 (Activation)   (None, None, None, 32)    0         
                                                                 
 max_pooling2d (MaxPooling2  (None, None, None, 32)    0         
 D)                                                              
                                                           

In [10]:
# define the feature extractor

# feature extractor input
feat_ext_input = gmp_model.input

# feature extractor output - do this at the ReLu activation layer - as this will give the same features as the dropout layer (It does not matter if it is the dropout or the activation layer, the extracted features will be the same)
feat_ext_output = gmp_model.layers[-4].output

feature_extractor_model = tf.keras.models.Model(inputs = feat_ext_input, outputs = feat_ext_output)

In [11]:
feature_extractor_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 conv2d (Conv2D)             (None, None, None, 32)    896       
                                                                 
 activation (Activation)     (None, None, None, 32)    0         
                                                                 
 conv2d_1 (Conv2D)           (None, None, None, 32)    9248      
                                                                 
 activation_1 (Activation)   (None, None, None, 32)    0         
                                                                 
 max_pooling2d (MaxPooling2  (None, None, None, 32)    0         
 D)                                                              
                                                             

In [12]:
# We need to preprocess all the data for the rest of the blocks, for the task of prediction and performance evaluation.

# The problem we currently have is the written code takes up a lot of time. -It takes too much time to do the feature extraction. Currently we are using the model

In [13]:
# Okay, let's try and solve this problem with the first block in the test dataset

Block 0103

In [14]:
blk_0103_data_path = '../../Spring_2024/S_lab_TasselNet/Block_3_TN/Block_3_images_and_xml'

In [15]:
# we can still use the function above to get the horizontal images
blk_0103_horizontal_images = get_horizontal_images(blk_0103_data_path)

In [16]:
# separate the inputs and the targets
blk_0103_input_time_point_images = blk_0103_horizontal_images[:13]
blk_0103_input_time_point_images.sort()
blk_0103_output_time_point_images = blk_0103_horizontal_images[-7:]
blk_0103_output_time_point_images.sort()

In [17]:
# blk_0103_input_time_point_images, blk_0103_output_time_point_images

In [18]:
# Okay, now we need to modify the next function a little to see if the timing for the computations can be improved

In [19]:
# What if we first stack the subwindows in a function, and then do model.predict may be in a separate function

In [20]:
def create_sub_windows(folder_path, file, stride = 30, kernel_size = 30):
    # joined image path
    joined_im_path = os.path.join(folder_path, file)
    # read the image
    loaded_im_file = plt.imread(joined_im_path)
    # create subwindows and get prediction
    img_height = loaded_im_file.shape[0]
    img_width = loaded_im_file.shape[1]

    # catch all subwindows here
    all_subwindows = []
    # you can also keep track the subwindows here if required - but let's not worry about that for now
    for i in  range(0, img_height, stride):
        for j in range(0, img_width, stride):
            sub_window = loaded_im_file[i: i + kernel_size, j : j + kernel_size,:]
            # resize the subwindow - for 300*300
            sub_window = resize(sub_window, (kernel_size, kernel_size,3))
            # append these to the list
            all_subwindows.append(sub_window)
            
    return all_subwindows  

In [21]:
# Try this for a single image

In [22]:
test_image_0 = blk_0103_input_time_point_images[0]

In [23]:
test_image_0_sub_windows = create_sub_windows(blk_0103_data_path, test_image_0, 30, 30)

In [24]:
# need to stack the list
test_image_0_sub_windows_stack = np.stack(test_image_0_sub_windows, axis = 0)

In [25]:
test_image_0_sub_windows_stack.shape

(910, 30, 30, 3)

In [26]:
%%time
# try model.predict with this
test_image_0_extracted_features = feature_extractor_model.predict(test_image_0_sub_windows_stack)

2025-06-04 12:54:35.650224: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907


29/29 [==============================] - 2s 6ms/step
CPU times: user 689 ms, sys: 217 ms, total: 906 ms
Wall time: 1.89 s


In [27]:
test_image_0_extracted_features.shape

(910, 32)

In [28]:
# Okay, so this happense pretty fast (in 2 seconds as opposed to 42 secods - so we do have a huge improvemet in execution time)

In [29]:
# Try doing this for all images

In [30]:
# input features for block 0103
blk_0103_input_time_point_images

['Block0103_2020_08_03.jpeg',
 'Block0103_2020_08_04.jpeg',
 'Block0103_2020_08_06.jpeg',
 'Block0103_2020_08_07.jpeg',
 'Block0103_2020_08_11.jpeg',
 'Block0103_2020_08_12.jpeg',
 'Block0103_2020_08_14.jpeg',
 'Block0103_2020_08_15.jpeg',
 'Block0103_2020_08_17.jpeg',
 'Block0103_2020_08_18.jpeg',
 'Block0103_2020_08_19.jpeg',
 'Block0103_2020_08_21.jpeg',
 'Block0103_2020_08_25.jpeg']

In [31]:
%%time
block_0103_extracted_features = []
for file in blk_0103_input_time_point_images:
    # get the subwindows
    subwindows = create_sub_windows(blk_0103_data_path, file, 30, 30)
    # stack the subwindows
    stacked_subwindows = np.stack(subwindows, axis = 0)
    # print the shape of this
    print(stacked_subwindows.shape)
    # extract features
    extracted_featrues = feature_extractor_model.predict(stacked_subwindows)
    # append the extracted features
    block_0103_extracted_features.append(extracted_featrues)

(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
(910, 30, 30, 3)
29/29 [==============================] - 0s 2ms/step
CPU times: user 4.46 s, sys: 148 ms, total: 4.61 s
Wall time: 4.37 s


In [32]:
block_0103_extracted_features[0].shape

(910, 32)

In [33]:
# now stack all these together?
blk_0103_stacked_input_features = np.stack(block_0103_extracted_features, axis = 1)

In [34]:
blk_0103_stacked_input_features.shape

(910, 13, 32)

In [35]:
# Okay, this seems to work really fast - better!
# now what if we do a function to get the predictions (extracted features too) - this would be easy for the rest of the blocks

#### WE NEED MORE SANITY CHECKS TO ENSURE WE ARE STACKING THE MATRICES CORRECTLY.

In [36]:
# For sanity check maybe use the already created and store np stacks